In [2]:
import os
from crewai import Agent, Task, Crew, Process
from langchain.tools import tool
#from langchain_community.tools import DuckDuckGoSearchAPIWrapper
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_community.llms import Ollama

# Initialize local LLMs using Ollama
research_llm = Ollama(model="deepseek-r1")  # Use DeepSeek R1 for research tasks
code_llm = Ollama(model="deepseek-coder")  # Use DeepSeek Coder for coding tasks

# Custom Tools (if needed)
class CustomTools:
    @tool("Save to File")
    def save_to_file(content: str, filename: str):
        """Saves content to a file."""
        with open(filename, "w") as file:
            file.write(content)
        return f"Content saved to {filename}"

# Initialize tools
search_tool = DuckDuckGoSearchAPIWrapper()  # Free search API
custom_tools = CustomTools()

# Define Agents
research_agent = Agent(
    role='Senior Research Analyst',
    goal='Find and analyze the most impactful academic papers',
    backstory="""You're a renowned research analyst with a knack for finding 
    cutting-edge papers in any field. You have access to all major academic 
    databases and can quickly assess paper quality.""",
    verbose=True,
    allow_delegation=False,
    tools=[search_tool.run],  # Use DuckDuckGo search tool
    llm=research_llm  # Use DeepSeek R1 for research
)

code_agent = Agent(
    role='Senior Software Engineer',
    goal='Generate high-quality code implementations from research methodologies',
    backstory="""You're a principal engineer at a top AI lab who specializes 
    in translating research papers into working implementations.""",
    verbose=True,
    allow_delegation=False,
    tools=[custom_tools.save_to_file],
    llm=code_llm  # Use DeepSeek Coder for coding
)

# Define Tasks
research_task = Task(
    description="""Find the best paper on {topic}. Extract its methodology section 
    and provide a detailed explanation of the approach.""",
    expected_output="A detailed methodology section from the best paper on the topic.",
    agent=research_agent,
    output_file='methodology.txt'
)

coding_task = Task(
    description="""Take the methodology from the research agent and generate Python code 
    that implements the described approach. Include test cases and documentation.""",
    expected_output="Complete Python implementation with example usage.",
    agent=code_agent,
    output_file='implementation.py'
)

# Create Crew
implementation_crew = Crew(
    agents=[research_agent, code_agent],
    tasks=[research_task, coding_task],
    verbose=2,
    process=Process.sequential
)

# Execute the workflow
if __name__ == "__main__":
    topic = input("Enter research topic: ")
    result = implementation_crew.kickoff(inputs={'topic': topic})
    print("\n\n########################")
    print("## Workflow Complete ##")
    print("########################")
    print(result)

C:\Users\samra\AppData\Local\Temp\ipykernel_9700\210870417.py:9: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  research_llm = Ollama(model="deepseek-r1")  # Use DeepSeek R1 for research tasks


ImportError: Could not import duckduckgo-search python package. Please install it with `pip install -U duckduckgo-search`.